In [ ]:
pip install datasets transformers

In [ ]:
pip install sacrebleu

In [ ]:
import pandas as pd
df = pd.read_csv('/content/overall_80K.csv')

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from transformers import DataCollatorForSeq2Seq
from datasets import load_metric


csv_path = "/content/overall_80K.csv"
df = pd.read_csv(csv_path)

texts = df['target'].tolist()
translations = df['source'].tolist()

train_size = int(0.9 * len(texts))
val_size = len(texts) - train_size

train_texts, val_texts = texts[:train_size], texts[train_size:]
train_translations, val_translations = translations[:train_size], translations[train_size:]

new_language_code = "<new_LNG>"

# 3. Создание кастомного датасета
source_lang = "<ru_XX>"
target_lang = "<new_LNG>"

class TranslationDataset(Dataset):
    def __init__(self, texts, translations, tokenizer, source_lang, target_lang):
        self.texts = texts
        self.translations = translations
        self.tokenizer = tokenizer
        self.source_lang = source_lang
        self.target_lang = target_lang

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        input_text = self.texts[idx]
        target_text = self.translations[idx]


        input_text = f"translate {self.source_lang} to {self.target_lang}: {input_text}"
        target_text = target_text


        inputs = self.tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=64)


        labels = self.tokenizer(target_text, return_tensors="pt", padding="max_length", truncation=True, max_length=64)


        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()


        labels = labels['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

train_dataset = TranslationDataset(train_texts, train_translations, tokenizer, source_lang, target_lang)
val_dataset = TranslationDataset(val_texts, val_translations, tokenizer, source_lang, target_lang)

# 4. Инициализация датаколлатора
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 5. Настройка тренера
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=64,
    learning_rate=5e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    save_steps=100,
    eval_steps=100,
    logging_dir="./logs",
    logging_steps=5,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# 6. Запуск тренировки
trainer.train()

# 7. Сохранение модели
model.save_pretrained("./finetuned-t5-small")
tokenizer.save_pretrained("./finetuned-t5-small")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
100,1.386000,1.610086
200,1.358300,1.597023
300,1.289100,1.587674
400,1.354900,1.572040
500,1.358700,1.565482
600,1.352300,1.557491
700,1.352700,1.549570
800,1.302100,1.542925
900,1.359700,1.538303
1000,1.243100,1.529787


('./finetuned-t5-small/tokenizer_config.json',
 './finetuned-t5-small/special_tokens_map.json',
 './finetuned-t5-small/spiece.model',
 './finetuned-t5-small/added_tokens.json')